In [8]:
from window_generator import WindowGenerator
import tensorflow as tf
import numpy as np
import pandas as pd
from read_data import *
from methods import *
MAX_EPOCHS = 20
VOICE = 2

In [9]:
def compile_and_fit(model, window, patience=2, verbose=0):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=patience,
                                                      mode='min',
                                                     restore_best_weights=True)
#  Early stopping when there is no improvement in loss. 
    model.compile(loss=tf.losses.MeanSquaredError(),
                  optimizer=tf.optimizers.Adam(),
                  metrics=[tf.metrics.MeanAbsoluteError()])

# At the end of each epoch, the model will iterate over the validation dataset and compute the validation loss and validation metrics.
    history = model.fit(window.train, epochs=MAX_EPOCHS,
                        validation_data=window.val,
                        callbacks=[early_stopping],
                        verbose=verbose)
    return history

In [10]:
multi_step_window = WindowGenerator(input_width=16, label_width=1, shift=1, batch_size=1, voice_number=VOICE)

for example_inputs, example_labels in multi_step_window.train.take(1):
    print(f"Inputs shape (batch, time, features): {example_inputs.shape}")
    print(f"Labels shape (batch, time, features): {example_labels.shape}")

print(multi_step_window.train)
# print(len(multi_step_window.label_columns_indices))

Inputs shape (batch, time, features): (1, 16, 6)
Labels shape (batch, time, features): (1, 1, 23)
<TakeDataset shapes: ((None, 16, 6), (None, 1, 23)), types: (tf.float32, tf.float32)>


In [4]:
n_output_nodes = len(multi_step_window.label_columns_indices)
linear_multi_step = tf.keras.Sequential([
    # Shape: (time, features) => (time*features)
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=n_output_nodes, activation='softmax'),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])
print('Input shape:', multi_step_window.example[0].shape)
print('Output shape:', linear_multi_step(multi_step_window.example[0]).shape)

# Train the model
history = compile_and_fit(linear_multi_step, multi_step_window)

print('Note that these are the LAST model metrics and not the BEST model metrics because of early stopping with restore_best_weights ')
print(
    "epochs: {}, loss: {:0.4f}, mean_absolute_error: {:0.4f}, val_loss: {:0.4f}, val_mean_absolute_error: {:0.4f}".format(
        len(history.history['loss']),
        history.history["loss"][-1],
        history.history["mean_absolute_error"][-1],
        history.history["val_loss"][-1],
        history.history["val_mean_absolute_error"][-1],
    )
)

# Evaluate the model
print("Evaluate")
result = linear_multi_step.evaluate(multi_step_window.test)
dict(zip(linear_multi_step.metrics_names, result))

Input shape: (1, 16, 6)
Output shape: (1, 1, 23)
Epoch 1/20
2664/2664 - 4s - loss: 0.0216 - mean_absolute_error: 0.0426 - val_loss: 0.0170 - val_mean_absolute_error: 0.0297
Epoch 2/20
2664/2664 - 3s - loss: 0.0154 - mean_absolute_error: 0.0288 - val_loss: 0.0148 - val_mean_absolute_error: 0.0253
Epoch 3/20
2664/2664 - 3s - loss: 0.0134 - mean_absolute_error: 0.0248 - val_loss: 0.0129 - val_mean_absolute_error: 0.0223
Epoch 4/20
2664/2664 - 3s - loss: 0.0125 - mean_absolute_error: 0.0231 - val_loss: 0.0127 - val_mean_absolute_error: 0.0210
Epoch 5/20
2664/2664 - 4s - loss: 0.0117 - mean_absolute_error: 0.0213 - val_loss: 0.0118 - val_mean_absolute_error: 0.0181
Epoch 6/20
2664/2664 - 3s - loss: 0.0110 - mean_absolute_error: 0.0195 - val_loss: 0.0108 - val_mean_absolute_error: 0.0170
Epoch 7/20
2664/2664 - 3s - loss: 0.0104 - mean_absolute_error: 0.0182 - val_loss: 0.0107 - val_mean_absolute_error: 0.0172
Epoch 8/20
2664/2664 - 3s - loss: 0.0100 - mean_absolute_error: 0.0173 - val_loss: 

{'loss': 0.00844827201217413, 'mean_absolute_error': 0.01319343876093626}

In [11]:
# Get last window
def predict_new_pitches(model, n=100):
    last_window = None
    last_pitch = None
    all_predicted_pitches = []
    all_probs = []

    for i in range(n):
        if last_window is None:
            for inputs, labels in multi_step_window.test_no_shuffle.as_numpy_iterator():
                pass

        #     print('window:', type(inputs),inputs ,'\n====\nlabels', labels)
        #     print('window shape:', inputs.shape, ' label shape:', labels.shape)

        # Model is expecting a shape of (batch, timestep, features) but for one sample this is (6,6)
        # Add dimension to input
        probabilities = model.predict(np.array([inputs]))[0]
        probabilities = probabilities[0]
        all_probs.append(probabilities)
        predicted_pitch = get_pitch_from_probability(
            probabilities,
            key=multi_step_window.pitch_conversion_key,
            method=SelectionMethod.WEIGHTED,
        )
        all_predicted_pitches.append(predicted_pitch)
        features = get_pitch_features(predicted_pitch)

        if last_pitch is None:
            last_pitch = get_voice(VOICE)[-1]

        # - find new duration based on previous and current pitch
        if last_pitch == predicted_pitch:
            duration = multi_step_window.df.iloc[-1]["dur"] + 1
        else:
            duration = 1

        last_pitch = predicted_pitch

        # create new dataframe
        new_df = pd.DataFrame(data={"dur": duration}, dtype=float, index=[0])
        (
            new_df["log_pitch"],
            new_df["chroma_x"],
            new_df["chroma_y"],
            new_df["c5_x"],
            new_df["c5_y"],
        ) = features
        # - Normalise features
        new_df = (new_df - multi_step_window.mean_df) / multi_step_window.std_df
        # - Make new window based on previous timesteps with a slide of 1 and the normalised features
        last_window = np.append(inputs[1:], new_df.values.tolist(), axis=0)
    return all_predicted_pitches, all_probs

In [12]:
predictions, probs = predict_new_pitches(linear_multi_step)
print(predictions)
print(probs[-10])
# wg = WindowGenerator(input_width=6, label_width=1, shift=1, batch_size=1)
# print(f'train {sum(1 for _ in wg.train)} val {sum(1 for _ in wg.val)} test {sum(1 for _ in wg.test)} ')
# print(f'full dataset {sum(1 for _ in wg.full_dataset)}')
# print(f'data size {wg.df.shape[0]} | file voice size: {len(get_voice(0))}')

# max(get_voice(2))

[54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54]
[4.32867148e-13 4.96175106e-12 4.64103941e-04 1.75748457e-10
 1.22951815e-09 6.39983817e-17 1.36209126e-11 2.87994400e-13
 9.69256549e-14 3.49328639e-06 2.23631709e-08 1.57546001e-05
 1.03520084e-04 4.92333857e-05 9.99359429e-01 4.37184333e-09
 3.35097479e-06 2.72856987e-13 6.52763399e-09 4.08965217e-09
 6.99336811e-10 7.07659694e-07 3.34397527e-07]
